In [1]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


## Q1 
Refer to HW3.ipynb and HW3_Q1.jpg

In [2]:
from neo4j import GraphDatabase

class Neo4JConnection:
    def __init__(self, uri, user, password):
        """Initialize the Neo4J connection with URI, username, and password."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        """Close the Neo4J connection."""
        if self.driver:
            self.driver.close()

    def execute_query(self, query, parameters=None):
        """Execute a Cypher query and return the result."""
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("ShoppingTrendsAnalysis").getOrCreate()

# Load CSV file into Spark DataFrame
file_path = '/Users/helenyu/Desktop/14677 SYS TL CH/HW3/shopping_trends.csv'
spark_df = spark.read.csv(file_path, header=True, inferSchema=True)

descriptive_stats = spark_df.describe()
descriptive_stats.show()

spark_df.show()

24/09/25 22:21:58 WARN Utils: Your hostname, Helens-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.206 instead (on interface en0)
24/09/25 22:21:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 22:21:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/25 22:22:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+------+--------------+-----------+---------------------+--------+----+------+------+------------------+-------------------+--------------+--------------+----------------+---------------+------------------+------------------------+----------------------+
|summary|       Customer ID|              Age|Gender|Item Purchased|   Category|Purchase Amount (USD)|Location|Size| Color|Season|     Review Rating|Subscription Status|Payment Method| Shipping Type|Discount Applied|Promo Code Used|Previous Purchases|Preferred Payment Method|Frequency of Purchases|
+-------+------------------+-----------------+------+--------------+-----------+---------------------+--------+----+------+------+------------------+-------------------+--------------+--------------+----------------+---------------+------------------+------------------------+----------------------+
|  count|              3900|             3900|  3900|          3900|       3900|                 390

In [4]:
uri = "neo4j+s://bc95cfdc.databases.neo4j.io"
user = "neo4j"
password = "bMochkdK8l-zAHJvqGNG1qxEg_zqoK72eZTJD5Vb7Gw"
conn = Neo4JConnection(uri, user, password)

## Q2
Refer to HW3.ipynb and HW3_Q2.jpg

In [6]:
with conn.driver.session() as session:
    for row in spark_df.collect():
        customer = row["Customer ID"]
        age = row["Age"]
        item = row["Item Purchased"]
        location = row["Location"]
        season = row["Season"]
        shipping = row["Shipping Type"]

        # Modify the query to merge relationships to prevent duplicates
        query = (
            "MERGE (c:Customer {name: $customer, age: $age, location: $location}) "
            "MERGE (i:Item {name: $item}) "
            "MERGE (c)-[r:BOUGHT]->(i) "  # Use MERGE to prevent duplicate relationships
            "ON CREATE SET r.season = $season, r.shipping = $shipping "
            "ON MATCH SET r.season = $season, r.shipping = $shipping "
        )

        params = {
            "customer": customer,
            "age": age,
            "location": location,
            "item": item,
            "season": season,
            "shipping": shipping
        }

        session.run(query, params)


## Q3
Refer to HW3.ipynb and 

In [18]:
def find_cus_over_age_50(connection):
    query = """
    MATCH (u:Customer)
    WITH count(u) AS total_users, count(CASE WHEN u.age >= 50 THEN 1 END) AS users_50_plus
    RETURN (toFloat(users_50_plus) / total_users) * 100 AS percentage_50_plus
     """
    result = connection.execute_query(query)
    percentage_50_plus = result[0]['percentage_50_plus']
    print(f"The percentage of overall customers over age of 50 is {percentage_50_plus:.2f}%")

## Q4
Refer to HW3.jpynb and HW3_Q4.jpg

In [20]:

def find_most_purchase_hawaii(connection):
    # Cypher query to find the most purchased item in Hawaii
    query = """
    MATCH (u:Customer {location: 'Hawaii'})-[:BOUGHT]->(i:Item)
    WITH i.name AS item_name, count(*) AS purchase_count
    RETURN item_name, purchase_count
    ORDER BY purchase_count DESC
    LIMIT 1
    """
    
    result = connection.execute_query(query)
    item_name = result[0]['item_name']
    purchase_count = result[0]['purchase_count']
    print(f"The most purchased item in Hawaii is: {item_name} with {purchase_count} purchases.")


## Q5
Refer to HW3.ipynb and HW3_Q5.jpg


In [9]:
def find_most_popular_season(connection):
    query = """
    MATCH (u:Customer)-[r:BOUGHT]->(i:Item)
    WITH r.season AS season, count(*) AS purchase_count
    RETURN season, purchase_count
    ORDER BY purchase_count DESC
    LIMIT 1
    """
    
    result = connection.execute_query(query)
    
    # Extract the season from the result
    if result:
        most_popular_season = result[0]['season']
        print(f"The most popular season to shop in is: {most_popular_season}")
        return most_popular_season
    else:
        print("No season data available.")
        return None
    

def find_most_popular_shipping_method(connection, season):
    query = """
    MATCH (u:Customer)-[r:BOUGHT {season: $season}]->(i:Item)
    WITH r.shipping AS shipping_method, count(*) AS shipping_count
    RETURN shipping_method, shipping_count
    ORDER BY shipping_count DESC
    LIMIT 1
    """
    
    result = connection.execute_query(query, {"season": season})
    
    # Extract the shipping method from the result
    if result:
        most_popular_shipping = result[0]['shipping_method']
        print(f"The most popular shipping method in {season} is: {most_popular_shipping}")
    else:
        print(f"No shipping data found for the season {season}.")


## Q6
Refer to HW3.ipynb and HW3_Q6.jpg

In [10]:
def recommend_popular_items_by_state(connection, state, limit=3):

    query = """
    MATCH (u:Customer {location: $state})-[:BOUGHT]->(i:Item)
    WITH i.name AS item_name, count(*) AS purchase_count
    RETURN item_name, purchase_count
    ORDER BY purchase_count DESC
    LIMIT $limit
    """
    
    result = connection.execute_query(query, {"state": state, "limit": limit})
    
    # Extract and print the recommended items
    if result:
        print(f"Top {len(result)} popular items in {state}:")
        for record in result:
            item_name = record["item_name"]
            purchase_count = record["purchase_count"]
            print(f"- {item_name} (purchased {purchase_count} times)")
    else:
        print(f"No purchase data found for {state}.")


In [19]:
# Main function
def main():

   uri = "neo4j+s://bc95cfdc.databases.neo4j.io"
   user = "neo4j"
   password = "bMochkdK8l-zAHJvqGNG1qxEg_zqoK72eZTJD5Vb7Gw"
   neo4j_conn = Neo4JConnection(uri, user, password)

   try:
      find_cus_over_age_50(neo4j_conn)
      find_most_purchase_hawaii(neo4j_conn)
      find_most_popular_season(neo4j_conn)
      most_popular_season = find_most_popular_season(neo4j_conn)
      find_most_popular_shipping_method(neo4j_conn, most_popular_season)
      recommend_popular_items_by_state(neo4j_conn, "Kentucky")
   finally:
 # Close the connection when done
      neo4j_conn.close()

if __name__ == "__main__":
   main()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n    MATCH (u:Customer)\n    WITH count(u) AS total_users, count(CASE WHEN u.age >= 50 THEN 1 END) AS users_50_plus\n    RETURN (toFloat(users_50_plus) / total_users) * 100 AS percentage_50_plus\n     '


The percentage of overall customers over age of 50 is 39.97%
The most purchased item in Hawaii is: Handbag with 6 purchases.
The most popular season to shop in is: Spring
The most popular season to shop in is: Spring
The most popular shipping method in Spring is: 2-Day Shipping
Top 3 popular items in Kentucky:
- Sweater (purchased 7 times)
- Handbag (purchased 7 times)
- Shirt (purchased 6 times)
